# Atelier de modélisation du métabolisme

### Maxime Mahout

## 1) Qu'est-ce que le métabolisme ?

L'ensemble des réactions qui se déroulent au sein d'une cellule (source image: Wikipedia).

![img](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Metabolic_Metro_Map.svg/800px-Metabolic_Metro_Map.svg.png)

### → Réseaux métaboliques:
* noeuds: métabolites
* arêtes: réaction
* pondération: stoechiométrie

\begin{equation}
\text{ex: } 2~ADP \rightarrow ATP + AMP
\end{equation}

## 2) Qu'est-ce qu'un modèle du métabolisme ?

Exemple: Mahout, Schwartz, Attal, Bakkar, Peres, 2024:

https://doi.org/10.1371/journal.pone.0313962

![img](https://journals.plos.org/plosone/article/figure/image?size=large&id=10.1371/journal.pone.0313962.g007)

Un modèle en sciences est une réduction d'un objet réel en une maquette que l'on peut analyser pour faire des prédictions que l'on espère avérées sur cet objet réel.

Ici, par exemple nous faisons des hypothèses sur la formation du stroma tumoral à partir d'un modèle réduit du métabolisme principal de la cellule cancéreuse.

## 3) Pourquoi modéliser le métabolisme ?

Comme en bio-statistique, on utilise les modèles afin de répondre à des questions scientifiques pour lesquelles on a formulé des hypothèses.

Egalement nous pouvons partir dépourvus d'une hypothèse et tester des propriétés sur des modèles pré-existants en espérant faire émerger des découvertes.


## 4) Comment modéliser le métabolisme ?

Il existe plusieurs façons de modéliser le métabolisme et sa complexité. Nous pouvons utiliser des approches 1) dynamiques ou 2) à l'état stationnaire pour modéliser les réactions biochimiques.

Les méthodes à l'état stationnaire, c'est à dire où la concentration en métabolite ne varie pas au cours du temps, telles que Flux Balance Analysis, sont les plus faciles d'accès.

Plus globalement afin d'obtenir des prédictions précises les modèles doivent respecter les principes de la "biologie des systèmes", une science qui s'attèle à représenter le système biologique dans toute sa complexité.

Autrement dit, le modèle biologique doit pouvoir intégrer le système "cellule" à travers tous ses niveaux: gène, ADN, ARN, molécules, chimie, protéine, enzyme, lipides, etc. et pourra être confirmé par tout autant d'analyses, génomique, transcriptomique, protéomique, métabolomique, etc. cf. https://www.nature.com/articles/nrm1857

## 5) Avec quelle méthode commencer ?

Je préconise la méthode "[Flux Balance Analysis](https://doi.org/10.1038/nbt.1614)" (FBA) et de partir "hypothesis-free" sur des modèles "à l'échelle du génome" préexistants, tels que ceux présents sur la base de données [BiGG](http://bigg.ucsd.edu/). Un modèle dit à l'échelle du génome est reconstruit à partir d'un génome et associe gènes aux réactions.

Afin de tester cela, nous pouvons installer l'outil `cobrapy` [(documentation)](https://cobrapy.readthedocs.io/) et tester le modèle par défaut `e_coli_core` (cf. [reference](https://europepmc.org/article/med/26443778)). 

Nous utiliserons `escher` [(documentation)](https://escher.readthedocs.io/en/latest/escher-python.html) pour la visualisation des réseaux métaboliques.

In [1]:
!pip list | grep -e "cobra" -e "Escher"

cobra                    0.26.2
Escher                   1.7.3


In [2]:
!jupyter nbextension list

Known nbextensions:
  config dir: /home/maxime/anaconda3/envs/escher-viz/etc/jupyter/nbconfig
    notebook section
      escher/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
  config dir: /home/maxime/.jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
  config dir: /home/maxime/.local/etc/jupyter/nbconfig
    notebook section
      nbextensions_configurator/config_menu/main  enabled 
      - Validating: problems found:
        - require?  X nbextensions_configurator/config_menu/main
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
    tree section
      nbextensions_configurator/tree_tab/main  enabled 
      - Validating: problems found:
        - require?  X nbextensions_configurator/tree_tab/main
  config dir: /etc/jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK


Remarquer ici que l'extension escher/extension pour les Jupyter notebooks est activée.

## 6) Visualisons un modèle métabolique

Le modèle `e_coli_core` est un modèle simplifié du métabolisme central de la bactérie *Escherichia coli*.

In [3]:
import escher
from escher import Builder
import cobra

In [4]:
!wget -nc http://bigg.ucsd.edu/static/models/e_coli_core.json
!wget -nc http://bigg.ucsd.edu/static/models/e_coli_core.xml

Fichier «e_coli_core.json» déjà présent ; pas de récupération.

Fichier «e_coli_core.xml» déjà présent ; pas de récupération.



In [5]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [6]:
builder

Builder()

La croissance de la bactérie est modélisée par une réaction appelée `BIOMASS_Ecoli_core`, réaction de synthèse de la biomasse. Cette réaction est calculée afin que son flux en métabolites soit plus ou moins proportionnelle au taux de croissance expérimental.

Ce modèle est donc adapté pour la méthode Flux Balance Analysis (FBA), c'est à dire l'analyse à l'état stationnaire des flux du réseau de réactions, à l'aide d'une optimisation linéaire avec comme fonction objective la croissance de la cellule.

## 7) Visualisons l'effet d'une mutation d'un gène sur le métabolisme

### a) Quel taux de croissance pour la bactérie *E. coli* wild type ?

In [7]:
from copy import copy

In [8]:
growth_rates = {}

In [9]:
model = cobra.io.read_sbml_model('e_coli_core.xml')

In [10]:
fluxes = model.optimize()

In [11]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [12]:
builder.reaction_data = fluxes.fluxes

In [13]:
builder

Builder(reaction_data={'PFK': 7.477381962160285, 'PFL': 0.0, 'PGI': 4.860861146496822, 'PGK': -16.023526143167…

In [14]:
fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

0.8739215069684302

In [15]:
growth_rates['WT'] = fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

Le taux de croissance simulé de la bactérie *E. coli* WT est de ~ 0.873 h⁻¹.

### b) Quel taux de croissance pour la bactérie *E. coli* avec knock-out du gène *pgi* ?

Voir la fiche UniProt: https://www.uniprot.org/uniprotkb/P0A6T1/entry.

In [16]:
model = cobra.io.read_sbml_model('e_coli_core.xml')

In [17]:
model.reactions.get_by_id('PGI') # bounds (-1000.0, 1000.0)

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x7fa8ac884460
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b4025
Lower bound,-1000.0
Upper bound,1000.0


In [18]:
for gene in list(model.reactions.get_by_id('PGI').genes):
    gene.knock_out() # b4025

In [19]:
model.reactions.get_by_id('PGI') # bounds (0, 0)

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x7fa8ac884460
Stoichiometry,g6p_c --> f6p_c D-Glucose 6-phosphate --> D-Fructose 6-phosphate
GPR,b4025
Lower bound,0
Upper bound,0


In [20]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [21]:
fluxes = model.optimize()

In [22]:
builder.reaction_data = fluxes.fluxes

In [23]:
builder

Builder(reaction_data={'PFK': 5.867064429485861, 'PFL': 0.0, 'PGI': 0.0, 'PGK': -14.431112198431936, 'PGL': 9.…

In [24]:
fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

0.8631595522084179

In [25]:
growth_rates['pgi-'] = fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

Le taux de croissance simulé de la bactérie *E. coli* avec *pgi*- est de ~ 0.8632 h⁻¹.

### c) Quel taux de croissance pour la bactérie *E. coli* avec knock-out du gène *zwf* (G6PDH) ?

Voir la fiche UniProt: https://www.uniprot.org/uniprotkb/P0AC53/entry.

In [26]:
model = cobra.io.read_sbml_model('e_coli_core.xml')

In [27]:
model.reactions.get_by_id('G6PDH2r') # bounds (-1000.0, 1000.0)

Reaction identifier,G6PDH2r
Name,Glucose 6-phosphate dehydrogenase
Memory address,0x7fa8acad2c20
Stoichiometry,"g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c D-Glucose 6-phosphate + Nicotinamide adenine dinucleotide phosphate <=> 6-phospho-D-glucono-1,5-lactone + H+ + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,b1852
Lower bound,-1000.0
Upper bound,1000.0


In [28]:
for gene in list(model.reactions.get_by_id('G6PDH2r').genes):
    gene.knock_out() # b1852

In [29]:
model.reactions.get_by_id('G6PDH2r') # bounds (0, 0)

Reaction identifier,G6PDH2r
Name,Glucose 6-phosphate dehydrogenase
Memory address,0x7fa8acad2c20
Stoichiometry,"g6p_c + nadp_c --> 6pgl_c + h_c + nadph_c D-Glucose 6-phosphate + Nicotinamide adenine dinucleotide phosphate --> 6-phospho-D-glucono-1,5-lactone + H+ + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,b1852
Lower bound,0
Upper bound,0


In [30]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [31]:
fluxes = model.optimize()

In [32]:
builder.reaction_data = fluxes.fluxes

In [33]:
builder

Builder(reaction_data={'PFK': 9.14076490103637, 'PFL': 0.0, 'PGI': 9.82291827155168, 'PGK': -17.70372507934551…

In [34]:
fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

0.8638133095040007

In [35]:
growth_rates['zwf-'] = fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

Le taux de croissance simulé de la bactérie *E. coli* avec *zwf*- est de ~ 0.8638 h⁻¹.

### d) Quel taux de croissance pour la bactérie *E. coli* avec knock-outs de *zwf* et *pwi* ?

In [39]:
model = cobra.io.read_sbml_model('e_coli_core.xml')

In [40]:
for gene in list(model.reactions.get_by_id('PGI').genes):
    gene.knock_out() # b4025

In [41]:
for gene in list(model.reactions.get_by_id('G6PDH2r').genes):
    gene.knock_out() # b1852

In [43]:
fluxes = model.optimize()

/home/maxime/anaconda3/envs/escher-viz/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [44]:
fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

0.0

In [45]:
growth_rates['pgi-, zwf-'] = fluxes.fluxes['BIOMASS_Ecoli_core_w_GAM']

Une absence de solution (problème "irrésoluble") traduit une impossibilité pour le modèle de produire de la biomasse, donc un flux de biomasse de zéro. 

## 7) Et si l'on intègre des données omiques ?

Reprenons la même expérience, mais au lieu d'utiliser de faire des "knock-out", supposons que nous avons des données omiques dans deux conditions.

Dans la première condition, C1, *zwf* est **sous-exprimé**. Les données de transcriptomiques et protéomiques valident cette hypothèse.

Dans la seconde condition, C2, *pgi* est **sous-exprimé**. Les données de transcriptomiques et protéomiques valident aussi cette hypothèse.

![img](https://raw.githubusercontent.com/maxm4/metabolic-modelling/refs/heads/main/img/expression.png)

L'enjeu ici est d'intégrer ces observations dans le modèle du métabolisme. Nous devons donc refléter cette donnée sur les bornes des flux des réactions.

La borne par défaut de toutes les réactions est (-1000.0, 1000.0) mais la borne d'EX_glc_e (-10.0, 0) force les valeurs de flux autour de l'intervalle [0.1, 10].

C1 se traduit donc par des bornes (-1.0, 1.0) au lieu de (-1000.0, 1000.0) pour G6PDH.

Et C2 se traduit donc par des bornes (-1.0, 1.0) au lieu de (-1000.0, 1000.0) pour PGI.